In [139]:
#导入所需库文件
from numpy import *
from functools import reduce
import re
import csv


In [140]:
# 创建数据集,加载数据
adClass = 1  # 广告,垃圾标识

def loadDataSet():
    '''加载数据集合及其对应的分类'''
    classVec = []    #0-1列表 第i个元素标识了wordList第i行类别
    wordList = []    #提取出的词矩阵，每一行是对应一个邮件的单词列表
    
    smss = open("./SMSSpamCollection.txt", 'r', encoding = 'utf-8')
    data = csv.reader(smss, delimiter = '\t')
    for line in data:      #line:左边一个"ham" or "spam"，右边一个大字符串
        if line[0] == "ham":
            classVec.append(0)
        else:
            classVec.append(1)
        wordList.append(textParse(line[1]))

    return wordList, classVec

In [141]:
def textParse(bigString):
    '''接收一个长字符串解析成字符串列表'''
    #将特殊符号作为划分标志进行字符串切分，即非字母，非数字
    listOfTokens = re.split(r'\W+', bigString)
    #除单字母  其它单词全变成小写
    return [tok.lower() for tok in listOfTokens if len(tok) > 2]

In [142]:
def doc2VecList(docList):
    """函数说明:数据进行并集操作，最后返回一个词不重复的并集"""
    #reduce(function, iterable[, initializer]): 从左至右积累地应用到 iterable 的条目，以便将该可迭代对象缩减为单一的值
    a = list(reduce(lambda x, y:set(x) | set(y), docList))
    return a  #['','',...,'']



In [143]:
def words2Vec(vecList, inputWords):     #所有词，输入的词组
    '''把单词转化为词向量'''
    dimensions = len(vecList)
    resultVec = [0] * dimensions
    for i in range(dimensions):
        if vecList[i] in inputWords:
            resultVec[i] += 1
    #转化为一维数组
    return array(resultVec)


In [144]:
def trainNB(trainMatrix, trainClass):
    """函数说明:计算，生成每个词对于类别上的概率"""
    # 类别行数
    numTrainClass = len(trainClass)
    # 列数
    numWords = len(trainMatrix[0])

    '''Laplacian +1平滑'''
    # 全部都初始化为1(全1数组)， 防止出现概率为0的情况
    p0Num = ones(numWords)
    p1Num = ones(numWords)
    # 相应的单词初始化为2
    p0Words = 2.0
    p1Words = 2.0

    # 统计每个分类的词的总数
    for i in range(numTrainClass):
        if trainClass[i] == 1:
            # 数组在对应的位置上相加
            p1Num += trainMatrix[i]
            p1Words += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Words += sum(trainMatrix[i])

    # 计算每种类型里面， 每个单词出现的概率
    # 在计算过程中，由于概率的值较小，于是取对数扩大数值域
    p0Vec = log(p0Num / p0Words)
    p1Vec = log(p1Num / p1Words)

    # 计算在类别中1出现的概率，0出现的概率可通过1-p得到
    pClass1 = sum(trainClass) / float(numTrainClass)
    return p0Vec, p1Vec, pClass1



In [145]:
def classifyNB(testVec, p0Vec, p1Vec, pClass1):
    """分类, 返回分类结果 0 or 1"""
    # 因为概率的值太小了，所以乘法改加法
    # 根据对数特性ln{ P(c)×P(X1|c)×P(X2|c)×...×P(Xn|c) } = lnP(c) + lnP(X1|c) + ... + lnP(Xn|c)
    # 可以简化计算且不失精度
    '''test * pVec已经在trainNB中取过对数了直接相加'''
    p1 = sum(testVec * p1Vec) + log(pClass1)
    p0 = sum(testVec * p0Vec) + log(1 - pClass1)
    if p0 > p1:
        return 0
    return 1



In [146]:
def printClass(words, testClass):
    if testClass == adClass:
        print(words, '推测为：广告邮件')
    else:
        print(words, '推测为：正常邮件')


In [147]:
def tNB():
    # 加载训练数据集
    docList, classVec = loadDataSet()    #单词矩阵、 01向量
    
    # 生成包含所有单词的list
    allWordsVec = doc2VecList(docList)
    
    # 构建词向量矩阵
    '''lambda中的x对应docList的每一行词组'''
    trainMat = list(map(lambda x : words2Vec(allWordsVec, x), docList))   #和docList对应的行(词)向量组
    
    # 训练计算每个词在分类上的概率
    # 其中p0V:每个单词在“非”分类出现的概率， p1V:每个单词在“是”分类出现的概率  pClass1：类别中是1的概率
    p0V, p1V, pClass1 = trainNB(trainMat, classVec)
    
    # 测试数据集
    text1 = "As a valued cutomer, I am pleased to advise you that following recent review of your Mob No"
    testwords1 = textParse(text1)
    testVec1 = words2Vec(allWordsVec, testwords1)
    # 通过将单词向量testVec代入，根据贝叶斯公式，比较各个类别的后验概率，判断当前数据的分类情况
    testClass1 = classifyNB(testVec1, p0V, p1V, pClass1)
    # 打印出测试结果
    printClass(testwords1, testClass1)
    
    text2 = "Please don't text me anymore. I have nothing else to say"
    testwords2 = textParse(text2)
    testVec2 = words2Vec(allWordsVec, testwords2)
    testClass2 = classifyNB(testVec2, p0V, p1V, pClass1)
    printClass(testwords2, testClass2)
    

In [148]:
if __name__ == '__main__':
    tNB()


['valued', 'cutomer', 'pleased', 'advise', 'you', 'that', 'following', 'recent', 'review', 'your', 'mob'] 推测为：广告邮件
['please', 'don', 'text', 'anymore', 'have', 'nothing', 'else', 'say'] 推测为：正常邮件
